# Rate of Turn Analysis

This notebook demonstrates how to analyze Rate of Turn (ROT) patterns in sailing data using NMEA logs.

In [ ]:
import os
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Import sailpolar modules
from sailpolar.parser.nmea0183 import NMEA0183Parser
from sailpolar.analysis.rot_analyzer import ROTAnalyzer
from sailpolar.analysis.rot_visualizer import ROTVisualizer

# Set up plotting
%matplotlib inline
plt.style.use('seaborn')

: 

## Configure Analysis Parameters

Set up the analysis parameters including file path and time windows:

In [1]:
# Analysis parameters
class Config:
    # File paths
    nmea_file_path = '../tests/data/Race-AIS-Sart-10m.txt'  # Update this path for your data
    output_dir = '../tests/output'
    
    # Time windows
    rot_window = timedelta(seconds=5)  # Window for ROT calculation
    stability_window = timedelta(minutes=5)  # Window for stability analysis
    
    # ROT thresholds (degrees/second)
    stable_threshold = 1.0  # Maximum ROT for 'stable' classification
    small_adjust_threshold = 3.0  # Maximum ROT for 'small adjustments'
    deliberate_turn_threshold = 10.0  # Maximum ROT for 'deliberate turn'

# Create output directory if it doesn't exist
os.makedirs(Config.output_dir, exist_ok=True)

NameError: name 'timedelta' is not defined

## Load and Parse NMEA Data

Load the NMEA data from file:

In [ ]:
def load_nmea_data(file_path):
    """Load and parse NMEA data from file"""
    parser = NMEA0183Parser()
    
    print(f"Loading data from {file_path}...")
    parsed_segments, _ = parser.parse_file(file_path)
    print(f"Loaded {len(parsed_segments)} data segment(s)")
    
    return parsed_segments

# Load the data
parsed_segments = load_nmea_data(Config.nmea_file_path)

## Analyze Rate of Turn

Process the data to calculate ROT and identify patterns:

In [ ]:
def analyze_rot(segments, window_size):
    """Analyze ROT patterns in the data segments"""
    rot_analyzer = ROTAnalyzer(window_size)
    all_rot_data = []
    
    for i, segment in enumerate(segments):
        print(f"Processing segment {i+1}/{len(segments)}")
        rot_data = rot_analyzer.analyze_segment(segment)
        
        if not rot_data.empty:
            rot_data['segment'] = i
            all_rot_data.append(rot_data)
    
    return pd.concat(all_rot_data, ignore_index=True) if all_rot_data else pd.DataFrame()

# Analyze the data
combined_rot_data = analyze_rot(parsed_segments, Config.rot_window)

# Display basic statistics
print("\nROT Pattern Distribution:")
pattern_counts = combined_rot_data['pattern'].value_counts()
for pattern, count in pattern_counts.items():
    percentage = (count / len(combined_rot_data)) * 100
    print(f"{pattern}: {count} samples ({percentage:.1f}%)")

print("\nROT Statistics:")
display(combined_rot_data['rot'].describe())

## Visualize Results

Create various visualizations of the ROT patterns:

In [ ]:
def create_visualizations(data, stability_window):
    """Create and display all ROT visualizations"""
    visualizer = ROTVisualizer()
    
    # Create all plots with save paths
    plots = [
        ("Time Series", "time_series.png", visualizer.plot_time_series),
        ("Pattern Distribution", "pattern_distribution.png", visualizer.plot_pattern_distribution),
        ("Pattern Transitions", "pattern_transitions.png", visualizer.plot_pattern_transitions),
        ("Rose Diagram", "rose_diagram.png", visualizer.plot_rose_diagram)
    ]
    
    for title, filename, plot_func in plots:
        print(f"\nGenerating {title}...")
        save_path = os.path.join(Config.output_dir, filename)
        plot_func(data)
        plt.savefig(save_path)
        plt.close()
        print(f"Saved to {save_path}")
    
    # Stability analysis (returns data)
    print("\nGenerating Stability Analysis...")
    save_path = os.path.join(Config.output_dir, "stability_analysis.png")
    stability_data = visualizer.plot_stability_analysis(data, window_size=stability_window)
    plt.savefig(save_path)
    plt.close()
    print(f"Saved to {save_path}")
    
    return stability_data

# Create visualizations
stability_data = create_visualizations(combined_rot_data, Config.stability_window)

## Analyze Stable Periods

Detailed analysis of stable patterns in the data:

In [ ]:
def analyze_stable_periods(data):
    """Analyze characteristics of stable periods"""
    stable_periods = data[data['pattern'] == 'STABLE'].copy()
    
    # Calculate stable period statistics
    stats = {
        'count': len(stable_periods),
        'total_duration': stable_periods['duration'].sum(),
        'mean_duration': stable_periods['duration'].mean(),
        'max_duration': stable_periods['duration'].max(),
        'median_duration': stable_periods['duration'].median()
    }
    
    # Print statistics
    print("Stable Period Analysis:")
    print(f"Total stable periods: {stats['count']}")
    print(f"Total time in stable state: {stats['total_duration']:.1f} seconds")
    print(f"Average period duration: {stats['mean_duration']:.2f} seconds")
    print(f"Median period duration: {stats['median_duration']:.2f} seconds")
    print(f"Longest stable period: {stats['max_duration']:.2f} seconds")
    
    # Plot duration distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(data=stable_periods, x='duration', bins=50)
    plt.title('Distribution of Stable Period Durations')
    plt.xlabel('Duration (seconds)')
    plt.ylabel('Count')
    
    # Save plot
    save_path = os.path.join(Config.output_dir, "stable_period_distribution.png")
    plt.savefig(save_path)
    plt.close()
    print(f"\nSaved duration distribution plot to {save_path}")
    
    return stats

# Analyze stable periods
stable_stats = analyze_stable_periods(combined_rot_data)

## Export Analysis Results

Save the analysis results to files:

In [ ]:
def export_results(rot_data, stability_data):
    """Export analysis results to CSV files"""
    # Export ROT analysis
    rot_file = os.path.join(Config.output_dir, "rot_analysis.csv")
    rot_data.to_csv(rot_file, index=False)
    print(f"Exported ROT analysis to {rot_file}")
    
    # Export stability analysis
    stability_file = os.path.join(Config.output_dir, "stability_analysis.csv")
    stability_data.to_csv(stability_file)
    print(f"Exported stability analysis to {stability_file}")

# Export all results
export_results(combined_rot_data, stability_data)

## Interactive Data Exploration

Use this section to interactively explore specific aspects of the data:

In [ ]:
# Example: Find longest stable periods
longest_stable = combined_rot_data[combined_rot_data['pattern'] == 'STABLE'].nlargest(5, 'duration')
print("Top 5 longest stable periods:")
display(longest_stable[['timestamp', 'heading', 'rot', 'duration']])

# Example: Pattern transitions
print("\nPattern transition probabilities:")
transitions = pd.crosstab(combined_rot_data['pattern'].shift(), 
                         combined_rot_data['pattern'], 
                         normalize='index')
display(transitions.round(3))